In [1]:
# 1️⃣ Hospital & department capacity setup (UKE-focused)

import random

# --- ICU units (real counts) -----------------------------------------------
# Each ICU must reserve 1 bed for resuscitation => operational_beds = total_beds - 1
ICU_TOTAL_BEDS = {
    "neurochirurgical_icu": 12,
    "neurological_icu": 12,
    "interdis_stage1": 12,     # UKE 1C
    "interdis_stage2": 12,     # UKE 1D
    "interdis_stage3": 12,     # UKE 1E
    "surgical_icu": 12,        # UKE 1F
    "internal_medicine_icu": 12,  # UKE 1G
    "cardio_icu": 12,          # UKE H1b (cardiology)
    "cardio_surgery_icu": 12,  # UKE H1b (cardiac surgery)
    "vascular_cardiac_icu": 8  # UKE H2b
}

ICU_OPERATIONAL_BEDS = {
    k: max(v - 1, 0) for k, v in ICU_TOTAL_BEDS.items()
}

# --- Non-ICU departments (simulated sizes) ----------------------------------
# Rule of thumb: Large=50, Medium=30, Small=20
DEPARTMENTS_SIZE = {
    # Large
    "internal_medicine": 50,
    "neurology": 50,
    "surgery_general": 50,

    # Medium
    "cardiology": 30,
    "pulmonology": 30,
    "gastroenterology": 30,
    "orthopedics": 30,
    "surgery_trauma": 30,
    "vascular_surgery": 30,

    # Small
    "oncology": 20,
    "pediatrics": 20,
    "obstetrics_gynecology": 20,
    "psychiatry": 20,
    "palliative": 20,
    "imc_internal_medicine": 20,
}

# Combine into a single capacity registry (operational bed counts)
OPER_BEDS = {
    **ICU_OPERATIONAL_BEDS,
    **DEPARTMENTS_SIZE
}

# --- Availability sampling ---------------------------------------------------
# Availability is sampled per patient/event: 0%..50% free, with a small chance of 0% hard block
def sample_availability_fraction(hard_block_prob: float = 0.10) -> float:
    """Return a fraction of free capacity in [0.0, 0.5]. Sometimes forced to 0.0."""
    if random.random() < hard_block_prob:
        return 0.0
    return random.uniform(0.0, 0.5)

def capacity_snapshot_normalized(hard_block_prob: float = 0.10) -> dict:
    """
    Produce a dict of normalized capacity scores per department (0..1),
    computed as available_beds / operational_beds for the current moment.
    """
    snap = {}
    for dept, oper_beds in OPER_BEDS.items():
        if oper_beds <= 0:
            snap[dept] = 0.0
            continue
        frac = sample_availability_fraction(hard_block_prob)
        available = int(round(oper_beds * frac))
        snap[dept] = available / oper_beds
    return snap

# Convenience lists if needed later
ICU_DEPARTMENTS = list(ICU_OPERATIONAL_BEDS.keys())
NON_ICU_DEPARTMENTS = list(DEPARTMENTS_SIZE.keys())
ALL_DEPARTMENTS = list(OPER_BEDS.keys())

# Sanity print (optional)
print("ICU operational beds:", ICU_OPERATIONAL_BEDS)
print("Non-ICU sizes:", DEPARTMENTS_SIZE)


ICU operational beds: {'neurochirurgical_icu': 11, 'neurological_icu': 11, 'interdis_stage1': 11, 'interdis_stage2': 11, 'interdis_stage3': 11, 'surgical_icu': 11, 'internal_medicine_icu': 11, 'cardio_icu': 11, 'cardio_surgery_icu': 11, 'vascular_cardiac_icu': 7}
Non-ICU sizes: {'internal_medicine': 50, 'neurology': 50, 'surgery_general': 50, 'cardiology': 30, 'pulmonology': 30, 'gastroenterology': 30, 'orthopedics': 30, 'surgery_trauma': 30, 'vascular_surgery': 30, 'oncology': 20, 'pediatrics': 20, 'obstetrics_gynecology': 20, 'psychiatry': 20, 'palliative': 20, 'imc_internal_medicine': 20}


In [2]:
# 2️⃣ Patient generator using real ICU and simulated non-ICU capacities

import uuid
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Fixed suspected conditions for now
SUSPECTED_CONDITIONS = [
    "cardiac", "respiratory", "neurological",
    "gastrointestinal", "oncology", "infection", "trauma"
]

HOME_TYPES = ["private", "nursing_home"]

def generate_patient():
    # Static patient attributes
    patient_id = str(uuid.uuid4())[:8]
    age = np.random.randint(18, 95)
    home_type = np.random.choice(HOME_TYPES)
    suspected_condition = np.random.choice(SUSPECTED_CONDITIONS)
    
    # EMS and ED timing
    ems_start_time = datetime(2025, 8, 7) + timedelta(
        hours=np.random.randint(0, 24),
        minutes=np.random.randint(0, 60)
    )
    ems_triage_code = np.random.randint(1, 5)  # 1=most urgent, 4=least
    ems_target_hospital = "UKE"
    ed_arrival_time = ems_start_time + timedelta(minutes=np.random.randint(5, 30))

    # Capacity snapshot for all departments (normalized)
    capacities = capacity_snapshot_normalized()

    # Target: for now, simple suitability score for ward assignment
    ward_assignment_suitability = np.random.uniform(0, 1)

    # Combine all fields
    record = {
        "patient_id": patient_id,
        "age": age,
        "home_type": home_type,
        "suspected_condition": suspected_condition,
        "ems_start_time": ems_start_time,
        "ems_triage_code": ems_triage_code,
        "ems_target_hospital": ems_target_hospital,
        "ed_arrival_time": ed_arrival_time,
    }

    # Add department capacities as features
    for dept, cap in capacities.items():
        record[f"cap_{dept}"] = cap

    # Output target(s)
    record["ward_assignment_suitability"] = ward_assignment_suitability

    return record

def generate_dataset(n_patients=500):
    patients = [generate_patient() for _ in range(n_patients)]
    return pd.DataFrame(patients)

# Quick test
df = generate_dataset(5)
print(df.head())
print("\nShape:", df.shape)


  patient_id  age     home_type suspected_condition      ems_start_time  \
0   9b4b4391   58       private    gastrointestinal 2025-08-07 12:09:00   
1   de71dbe0   78  nursing_home         respiratory 2025-08-07 22:25:00   
2   602be59d   37  nursing_home             cardiac 2025-08-07 15:34:00   
3   4752efa0   29  nursing_home    gastrointestinal 2025-08-07 15:32:00   
4   df3f1d55   52  nursing_home            oncology 2025-08-07 07:16:00   

   ems_triage_code ems_target_hospital     ed_arrival_time  \
0                4                 UKE 2025-08-07 12:25:00   
1                2                 UKE 2025-08-07 22:30:00   
2                3                 UKE 2025-08-07 15:43:00   
3                4                 UKE 2025-08-07 15:59:00   
4                3                 UKE 2025-08-07 07:29:00   

   cap_neurochirurgical_icu  cap_neurological_icu  ...  cap_orthopedics  \
0                  0.272727              0.090909  ...         0.200000   
1                  0.27272

In [3]:
def generate_patient():
    patient_id = str(uuid.uuid4())[:8]

    # Demographics
    age = np.random.randint(0, 100)
    home_type = random.choice(['private', 'nursing_home'])

    # Clinical presentation
    suspected_condition = random.choice([
        'cardiac', 'respiratory', 'neurological',
        'infection', 'trauma', 'oncology', 'gastrointestinal'
    ])

    # EMS data
    ems_start_time = datetime(2025, 8, 7) + timedelta(
        hours=np.random.randint(0, 24),
        minutes=np.random.randint(0, 60)
    )
    ems_triage_code = np.random.choice([1, 2, 3, 4], p=[0.1, 0.2, 0.4, 0.3])
    ems_target_hospital = "UKE"  # fixed

    # ED arrival time
    travel_time_min = np.random.randint(5, 25)
    ed_arrival_time = ems_start_time + timedelta(minutes=travel_time_min)

    # Derived features
    day_of_week = ed_arrival_time.strftime("%A")
    hour_of_day = ed_arrival_time.hour

    # Capacities — ICU real values minus 1 bed for resus
    icu_capacities = {
        'neurochirurgical_icu': 11,
        'neurological_icu': 11,
        'interdis_stage1': 11,
        'interdis_stage2': 11,
        'interdis_stage3': 11,
        'surgical_icu': 11,
        'internal_medicine_icu': 11,
        'cardio_icu': 11,
        'cardio_surgery_icu': 11,
        'vascular_cardiac_icu': 7
    }

    # Simulate ICU availability between 0–100% of operational beds
    icu_caps = {f"cap_{k}": round(np.random.randint(0, beds + 1) / beds, 2)
                for k, beds in icu_capacities.items()}

    # Non-ICU sizes
    non_icu_sizes = {
        'internal_medicine': 50,
        'neurology': 50,
        'surgery_general': 50,
        'cardiology': 30,
        'pulmonology': 30,
        'gastroenterology': 30,
        'orthopedics': 30,
        'surgery_trauma': 30,
        'vascular_surgery': 30,
        'oncology': 20,
        'pediatrics': 20,
        'obstetrics_gynecology': 20,
        'psychiatry': 20,
        'palliative': 20,
        'imc_internal_medicine': 20
    }

    # Simulate availability 0–50% for non-ICU wards
    non_icu_caps = {f"cap_{k}": round(np.random.randint(0, int(beds * 0.5) + 1) / beds, 2)
                    for k, beds in non_icu_sizes.items()}

    # Output variable — ground truth
    ward_assignment_suitability = round(np.clip(np.random.normal(0.7, 0.15), 0, 1), 3)

    return {
        "patient_id": patient_id,
        "age": age,
        "home_type": home_type,
        "suspected_condition": suspected_condition,
        "ems_start_time": ems_start_time,
        "ems_triage_code": ems_triage_code,
        "ems_target_hospital": ems_target_hospital,
        "ed_arrival_time": ed_arrival_time,
        "day_of_week": day_of_week,
        "hour_of_day": hour_of_day,
        **icu_caps,
        **non_icu_caps,
        "ward_assignment_suitability": ward_assignment_suitability
    }


In [4]:
# 4️⃣ Generate dataset with expanded capacity features
def generate_dataset(n_patients=500):
    return [generate_patient() for _ in range(n_patients)]

df = pd.DataFrame(generate_dataset(n_patients=500))

# Basic hygiene
assert df.isna().sum().sum() == 0, "Found unexpected NaNs — check generator."
assert (df["ems_triage_code"].between(1, 4)).all(), "ems_triage_code out of expected bounds."

print("Shape:", df.shape)
print("Columns:", list(df.columns))

print("\nValue counts — suspected_condition:")
print(df["suspected_condition"].value_counts())

print("\nValue counts — home_type:")
print(df["home_type"].value_counts())

display(df.head(3))


Shape: (500, 36)
Columns: ['patient_id', 'age', 'home_type', 'suspected_condition', 'ems_start_time', 'ems_triage_code', 'ems_target_hospital', 'ed_arrival_time', 'day_of_week', 'hour_of_day', 'cap_neurochirurgical_icu', 'cap_neurological_icu', 'cap_interdis_stage1', 'cap_interdis_stage2', 'cap_interdis_stage3', 'cap_surgical_icu', 'cap_internal_medicine_icu', 'cap_cardio_icu', 'cap_cardio_surgery_icu', 'cap_vascular_cardiac_icu', 'cap_internal_medicine', 'cap_neurology', 'cap_surgery_general', 'cap_cardiology', 'cap_pulmonology', 'cap_gastroenterology', 'cap_orthopedics', 'cap_surgery_trauma', 'cap_vascular_surgery', 'cap_oncology', 'cap_pediatrics', 'cap_obstetrics_gynecology', 'cap_psychiatry', 'cap_palliative', 'cap_imc_internal_medicine', 'ward_assignment_suitability']

Value counts — suspected_condition:
suspected_condition
trauma              85
infection           77
gastrointestinal    73
cardiac             73
oncology            70
respiratory         66
neurological        

,patient_id,age,home_type,suspected_condition,ems_start_time,ems_triage_code,ems_target_hospital,ed_arrival_time,day_of_week,hour_of_day,...,cap_orthopedics,cap_surgery_trauma,cap_vascular_surgery,cap_oncology,cap_pediatrics,cap_obstetrics_gynecology,cap_psychiatry,cap_palliative,cap_imc_internal_medicine,ward_assignment_suitability
0,9d355a75,83,nursing_home,infection,2025-08-07 11:29:00,3,UKE,2025-08-07 11:34:00,Thursday,11,...,0.37,0.23,0.0,0.00,0.15,0.15,0.05,0.25,0.5,0.818
1,38bdef96,50,nursing_home,trauma,2025-08-07 19:34:00,4,UKE,2025-08-07 19:50:00,Thursday,19,...,0.30,0.43,0.4,0.15,0.15,0.50,0.35,0.00,0.1,0.632
2,57a73543,19,private,trauma,2025-08-07 01:40:00,3,UKE,2025-08-07 01:46:00,Thursday,1,...,0.27,0.30,0.4,0.00,0.45,0.25,0.30,0.30,0.1,0.606


In [5]:
# 5️⃣ Preprocessing setup for expanded capacity dataset

# Feature lists
FEATS_CAT = ["home_type", "suspected_condition", "day_of_week"]
FEATS_NUM = ["age", "ems_triage_code", "hour_of_day"] + \
            [c for c in df.columns if c.startswith("cap_")]

TARGET = "ward_assignment_suitability"

# Split
X = df[FEATS_CAT + FEATS_NUM]
y = df[TARGET]

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# Preprocessing pipeline
pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), FEATS_CAT),
    ("num", StandardScaler(), FEATS_NUM)
])


NameError: name 'train_test_split' is not defined

In [ ]:
# Ridge regression sanity check
import time
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

start_time = time.time()

# Preprocessor (re-using FEATS_CAT and FEATS_NUM from before)
pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), FEATS_CAT),
    ("num", StandardScaler(), FEATS_NUM)
])

model = Pipeline([
    ("pre", pre),
    ("reg", Ridge(alpha=1.0))
])

# Train
model.fit(X_train, y_train)

# Validate
y_val_pred = model.predict(X_val)
mse = mean_squared_error(y_val, y_val_pred)
mae = mean_absolute_error(y_val, y_val_pred)

end_time = time.time()

print(f"Validation MSE: {mse:.4f}")
print(f"Validation MAE: {mae:.4f}")
print(f"Training time: {end_time - start_time:.2f} seconds")


In [ ]:
# 🔹 Step 1: Preprocess with expanded feature set
FEATS_CAT = ["home_type", "suspected_condition", "day_of_week"]
FEATS_NUM = [
    "age", "ems_triage_code", "hour_of_day",
    # All capacity columns
] + [col for col in df.columns if col.startswith("cap_")]

TARGET = "ward_assignment_suitability"

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), FEATS_CAT),
    ("num", StandardScaler(), FEATS_NUM)
])

X_train_np = pre.fit_transform(X_train)
X_val_np   = pre.transform(X_val)
X_test_np  = pre.transform(X_test)

y_train_np = y_train.values.astype("float32")
y_val_np   = y_val.values.astype("float32")
y_test_np  = y_test.values.astype("float32")

print("Shapes:", X_train_np.shape, X_val_np.shape, X_test_np.shape)

# 🔹 Step 2: Convert to torch tensors and loaders
import torch
from torch.utils.data import TensorDataset, DataLoader

Xtr = torch.tensor(X_train_np, dtype=torch.float32)
ytr = torch.tensor(y_train_np.reshape(-1, 1), dtype=torch.float32)
Xva = torch.tensor(X_val_np, dtype=torch.float32)
yva = torch.tensor(y_val_np.reshape(-1, 1), dtype=torch.float32)
Xte = torch.tensor(X_test_np, dtype=torch.float32)
yte = torch.tensor(y_test_np.reshape(-1, 1), dtype=torch.float32)

train_loader = DataLoader(TensorDataset(Xtr, ytr), batch_size=64, shuffle=True)
val_loader   = DataLoader(TensorDataset(Xva, yva), batch_size=256)
test_loader  = DataLoader(TensorDataset(Xte, yte), batch_size=256)

input_dim = Xtr.shape[1]
print("Input dim:", input_dim)

# 🔹 Step 3: Define the MLP
import torch.nn as nn
import torch.optim as optim
import numpy as np

class MLP(nn.Module):
    def __init__(self, d_in):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in, 128), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.net(x)

mlp = MLP(input_dim)

criterion = nn.MSELoss()
optimizer = optim.Adam(mlp.parameters(), lr=1e-3)

# 🔹 Step 4: Training loop with early stopping
best_val = np.inf
patience, bad = 8, 0
epochs = 50

for ep in range(1, epochs + 1):
    mlp.train()
    train_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred = mlp(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
    train_loss /= len(train_loader.dataset)

    mlp.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            val_loss += criterion(mlp(xb), yb).item() * xb.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {ep:02d} | train MSE {train_loss:.4f} | val MSE {val_loss:.4f}")

    if val_loss + 1e-6 < best_val:
        best_val = val_loss
        bad = 0
        best_state = {k: v.cpu().clone() for k, v in mlp.state_dict().items()}
    else:
        bad += 1
        if bad >= patience:
            print("Early stopping.")
            break

mlp.load_state_dict(best_state)


In [ ]:
# Test set evaluation
mlp.eval()
with torch.no_grad():
    y_pred_test = np.vstack([mlp(xb).cpu().numpy() for xb, _ in test_loader]).ravel()

mse_test = mean_squared_error(y_test_np, y_pred_test)
mae_test = mean_absolute_error(y_test_np, y_pred_test)

print(f"MLP Test MSE: {mse_test:.4f} | MAE: {mae_test:.4f}")

# Scatter plot
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_test_np, y=y_pred_test, alpha=0.7)
plt.plot([0,1], [0,1], 'r--', label="Perfect")
plt.xlabel("Actual suitability")
plt.ylabel("Predicted suitability")
plt.title("MLP — Predicted vs Actual Suitability")
plt.legend()
plt.show()
